In [44]:
import nltk
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import matplotlib
from matplotlib import pyplot as plt
# import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
train = pd.read_csv("C:/Users/karan/OneDrive/Documents/UVA/Fall Semester/SYS 6018/sys6018-competition-/sys6018-competition-blogger-characteristics/karan/train.csv")

In [6]:
train=train.groupby(['user.id','age'])["text"].apply(lambda x: "%s" % ' '.join(x)).reset_index(name='text')

In [7]:
train.head()

,user.id,age,text
0,1,17,Doritos Fuel of Space urlLink : Ian My...
1,3,16,"Yeah, I set up this blog so Heather w..."
2,4,17,Not too much exciting has happened ...
3,5,47,European Pilgrimage For High sch...
4,6,16,Last night's dream is very blurry. T...


In [8]:
train_text = train[["user.id", "text", "age"]]

In [10]:
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)
train_text['text'] = train_text['text'].apply(remove_punctuation)

In [11]:
nltk.download('stopwords')
sw = stopwords.words('english')
def stopwords(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)
train_text['text'] = train_text['text'].apply(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
stemmer = SnowballStemmer("english")

def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 
train_text['text'] = train_text['text'].apply(stemming)

In [31]:
count_vect = CountVectorizer(max_features=200)
X_train_counts = count_vect.fit_transform(train_text.text)
X_train_counts.shape

(12880, 200)

In [32]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(12880, 200)

In [33]:
clf = MultinomialNB().fit(X_train_tfidf, train_text.age)

In [16]:
test = pd.read_csv("C:/Users/karan/OneDrive/Documents/UVA/Fall Semester/SYS 6018/sys6018-competition-/sys6018-competition-blogger-characteristics/karan/test.csv")

In [17]:
test=test.groupby('user.id')["text"].apply(lambda x: "%s" % ' '.join(x)).reset_index(name='text')

In [18]:
test_text = test[["user.id", "text"]]

In [19]:
test_text['text'] = test_text['text'].apply(remove_punctuation)

In [20]:
test_text['text'] = test_text['text'].apply(stopwords)

In [21]:
test_text['text'] = test_text['text'].apply(stemming)

In [34]:
X_new_counts = count_vect.transform(test_text.text)

In [35]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [36]:
predicted = clf.predict(X_new_tfidf)

In [37]:
cols = ["user.id", "age"]
test_output = pd.DataFrame(columns = cols)
test_output[["user.id"]] = test[["user.id"]]
test_output[["age"]] = predicted

In [38]:
test_output.head()

,user.id,age
0,2,17
1,8,23
2,9,17
3,10,23
4,11,23


In [39]:
# train.to_csv("train_FRE.csv", encoding = 'utf-8', index = False)
test_output.to_csv("test_sklearn_output.csv", encoding = 'utf-8', index = False)